In [1]:
!pip install fastapi Pillow python-multipart torch transformers uvicorn nest-asyncio pyngrok

In [2]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

In [3]:
# prepare image + question
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

text = "What are the colors of the cats?"

# prepare inputs
encoding = processor(image, text, return_tensors="pt")


In [4]:

# forward pass
outputs = model(**encoding)
logits = outputs.logits
idx = logits.argmax(-1).item()


print("Predicted answer:", model.config.id2label[idx])

# TODO: put above code into a function that accepts image and text as input

Predicted answer: brown and black


In [5]:
def model_pipeline(text: str, image: Image):
    # prepare inputs
    encoding = processor(image, text, return_tensors="pt")

    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()

    return  model.config.id2label[idx]

In [11]:
from fastapi import FastAPI, UploadFile, Form
import io
from PIL import Image

app = FastAPI()

@app.post("/ask")
async def ask(text: str = Form(...), image: UploadFile = None):
    # Read the uploaded image
    contents = await image.read()
    image_pil = Image.open(io.BytesIO(contents))

    # Run your model pipeline
    answer = model_pipeline(text, image_pil)
    return {"answer": answer}

In [12]:
import nest_asyncio
nest_asyncio.apply()
from pyngrok import ngrok
import uvicorn

ngrok.set_auth_token("34AKoAHoDhMN9tFg8ZHYglGGNGr_6qGnJoSYhM3A3jynTAUxe")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
config = uvicorn.Config(app, port=8000, log_level="info")
server = uvicorn.Server(config)

import asyncio
# Use asyncio to serve, compatible with nest_asyncio
await server.serve()

Public URL: https://scrofulously-matrilineal-giana.ngrok-free.dev


INFO:     Started server process [1047]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2600:4808:4078:d401:58e0:a6e4:eedd:ae4f:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     54.86.50.139:0 - "POST /ask HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1047]
